importing the relevant libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from scipy import stats

Exploraory data analysis 

In [4]:
df=pd.read_csv("data.csv")

In [5]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [6]:
df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [7]:
df['Gender'].dtype

dtype('O')

In [8]:
df.isna().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [9]:
# filling na values with mean value in loanAmount column
mean=np.mean(df["LoanAmount"])
df["LoanAmount"].fillna(mean,inplace=True)

/tmp/ipykernel_7682/3789532600.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["LoanAmount"].fillna(mean,inplace=True)


In [10]:
# filling na values with mode value in LoanAmount column
mode=df["Loan_Amount_Term"].mode()
df["Loan_Amount_Term"].fillna(512,inplace=True)

/tmp/ipykernel_7682/3378942044.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Loan_Amount_Term"].fillna(512,inplace=True)


In [11]:
# checking for outliers
cols_outliers =["ApplicantIncome","CoapplicantIncome", "LoanAmount","Loan_Amount_Term"]
        

In [12]:
# def outlier_detect(df,column):
#     outlier_column=[]
#     for x in  df[column]:
#         z=np.divide(x-np.mean(df[column]),np.std(df[column]))
#         if z > 3:
#             outlier_column.append(x)
#     print(outlier_column) 


In [13]:
# for column in cols_outliers:
#     outlier_detect(df,column)

In [14]:
# dropping Loan_ID since it no useful info
df.drop("Loan_ID", axis=1, inplace=True)
# converting all the values in credit history to object
df["Credit_History"].replace({0:"No",1:"yes",np.nan:"unknown"},inplace=True)
df["Loan_Status"].replace({"Y":1,"N":0}, inplace=True)

/tmp/ipykernel_7682/848080436.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Credit_History"].replace({0:"No",1:"yes",np.nan:"unknown"},inplace=True)
/tmp/ipykernel_7682/848080436.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(v

In [15]:
# independent variable
X=df.iloc[:,:-1]
# dependent variable
y=df.iloc[:,-1:]

In [16]:
X=pd.get_dummies(data=X, dummy_na=True)
X

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Gender_Female,Gender_Male,Gender_nan,Married_No,Married_Yes,Married_nan,...,Self_Employed_Yes,Self_Employed_nan,Credit_History_No,Credit_History_unknown,Credit_History_yes,Credit_History_nan,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Property_Area_nan
0,5849,0.0,146.412162,360.0,False,True,False,True,False,False,...,False,False,False,False,True,False,False,False,True,False
1,4583,1508.0,128.000000,360.0,False,True,False,False,True,False,...,False,False,False,False,True,False,True,False,False,False
2,3000,0.0,66.000000,360.0,False,True,False,False,True,False,...,True,False,False,False,True,False,False,False,True,False
3,2583,2358.0,120.000000,360.0,False,True,False,False,True,False,...,False,False,False,False,True,False,False,False,True,False
4,6000,0.0,141.000000,360.0,False,True,False,True,False,False,...,False,False,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,2900,0.0,71.000000,360.0,True,False,False,True,False,False,...,False,False,False,False,True,False,True,False,False,False
610,4106,0.0,40.000000,180.0,False,True,False,False,True,False,...,False,False,False,False,True,False,True,False,False,False
611,8072,240.0,253.000000,360.0,False,True,False,False,True,False,...,False,False,False,False,True,False,False,False,True,False
612,7583,0.0,187.000000,360.0,False,True,False,False,True,False,...,False,False,False,False,True,False,False,False,True,False


In [17]:
X.shape

(614, 29)

In [18]:
X.drop(["Education_nan","Credit_History_nan","Property_Area_nan",], axis=1, inplace=True)
X.shape

(614, 26)

In [19]:
scaler=MinMaxScaler()
X=pd.DataFrame(scaler.fit_transform(X),columns=X.columns)
y=y.values.flatten()


MODELING

In [21]:
# RandomForest=RandomForestClassifier(n_estimators=200,max_depth=3,)
# LogisticRegres=LogisticRegression()
# Svm=SVC(kernel='linear')
# models=[RandomForest,LogisticRegres,Svm]

# for model in models:
#     score=cross_val_score(model,X,y, scoring="f1",cv=5,n_jobs=-1)
#     print(np.mean(score))

# SVC is the best model

In [22]:
test_data=pd.read_csv("loan-test.csv")
test_data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


Cleaning and preparing the test data similar to the train data

In [24]:
test_df=test_data.iloc[:,1:].copy()
Loan_ID=test_data.iloc[:,0:1]

# filling the na values with the mean of the column
mean=np.mean(df["LoanAmount"])
test_df["LoanAmount"].fillna(mean,inplace=True)

# filling na values with mode value in LoanAmount column
mode=test_df["Loan_Amount_Term"].mode()
test_df["Loan_Amount_Term"].fillna(360,inplace=True)

test_df["Credit_History"].replace({0:"No",1:"yes",np.nan:"unknown"},inplace=True)
test_df=pd.get_dummies(data=test_df, dummy_na=True)
test_df.drop(["Education_nan","Credit_History_nan","Property_Area_nan",], axis=1, inplace=True)
test_df=pd.DataFrame(scaler.fit_transform(test_df),columns=test_df.columns)
test_df.head()


/tmp/ipykernel_7682/583274330.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df["LoanAmount"].fillna(mean,inplace=True)
/tmp/ipykernel_7682/583274330.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Gender_Female,Gender_Male,Gender_nan,Married_No,Married_Yes,Married_nan,...,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Self_Employed_nan,Credit_History_No,Credit_History_unknown,Credit_History_yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,0.078865,0.000000,0.157088,0.746835,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.042411,0.062500,0.187739,0.746835,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.068938,0.075000,0.344828,0.746835,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.032263,0.106083,0.137931,0.746835,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.045168,0.000000,0.095785,0.746835,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


Predicting the test data

In [26]:
Model=SVC(kernel='linear')
Model.fit(X,y)
prediction=Model.predict(test_df)
result=pd.DataFrame({"Loan_ID":Loan_ID.values.flatten(),"prediction":prediction,})

result.replace({1:"Y", 0:"N"},inplace=True)
result

,Loan_ID,prediction
0,LP001015,Y
1,LP001022,Y
2,LP001031,Y
3,LP001035,Y
4,LP001051,Y
...,...,...
362,LP002971,Y
363,LP002975,Y
364,LP002980,Y
365,LP002986,Y


In [ ]:

prediction=Model.predict(test_df)
result=pd.DataFrame({"Loan_ID":Loan_ID.values.flatten(),"prediction":prediction,})

result.replace({1:"Y", 0:"N"},inplace=True)
result